# Load env

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import importlib
from reports_etl import *
from enrich_holdings import *
from fossil_classification import *
from holdings_analysis import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [4]:
last_updated()

.py file last modified: 2022-09-28 17:22:01.137817


# Bulk download reports
Using the http response recorded while searching for reports here:
https://employersinfocmp.cma.gov.il/#/publicreports

In [5]:
# get reports from response.json search results
response_path = "data/downloaded reports/company reports/2022Q3/"
reports = get_reports_from_response(response_path)

Number of reports included in response.json: 66


## Check for missing reports

In [ ]:
# reports[
#     reports["DocumentId"] == 2308537
# ]
# reports.head()

In [ ]:
# filter specific reports for quick analysis
# reports = reports[reports["ParentCorpName"].str.startswith("מגדל")]

In [6]:
import glob
from pathlib import Path

# download missing files
downloaded = [Path(path).stem for path in glob.glob(response_path + "*.xls*")]
print("already downloaded {} out of {}".format(len(downloaded), len(reports)))

reports_to_download = reports[
    ~reports["DocumentId"].astype('str').isin(downloaded)
]

len(reports_to_download)

already downloaded 0 out of 66


66

In [7]:
# download reports - run only once per response
download_reports(reports_to_download, response_path, sleep=3)

In [8]:
reports.head()

,EntityId,CodeFile,DocumentCode,DocumentName,DocumentId,fileExt,ReportFor,ReportPeriodCode,ReportPeriodDesc,EntHistLevel,SystemCode,FunctionCode,LicenseCode,LicenseStatusId,DateStart,DateFinish,ProductNum,LegalId,Name,ShortName,EventDate,StatusDate,SystemName,ReportHistLevel,EventId,DocStatus,EntityName,LicenseId,ParentCorpName,InvestmentNumber,ParentCorpLegalId,ParentCorpEntlId,IsMaxRowsExceeded,url
0,270,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2329756,xlsx,1,71000001,2022 רבעון 3,0,300003,0,0,0,NaN,NaN,0,570011767,NaN,NaN,01/07/2022,27/10/2022,גמל,0,673490,1,NaN,0,קופת תגמולים של עובדי אל על נתיבי אוויר לישראל...,NaN,570011767,NaN,False,https://employersinfocmp.cma.gov.il/api/Public...
1,388,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2330602,xlsx,1,71000001,2022 רבעון 3,0,300003,0,0,0,NaN,NaN,0,520005497,NaN,NaN,01/07/2022,30/10/2022,גמל,0,673839,1,NaN,0,החברה לניהול קופות התגמולים והפיצויים של עובדי...,NaN,520005497,NaN,False,https://employersinfocmp.cma.gov.il/api/Public...
2,220,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2330952,xlsx,1,71000001,2022 רבעון 3,0,300003,0,0,0,NaN,NaN,0,570009449,NaN,NaN,01/07/2022,31/10/2022,גמל,0,673963,1,NaN,0,"עוצ""מ - אגודה שיתופית לניהול קופות גמל בע""מ",NaN,570009449,NaN,False,https://employersinfocmp.cma.gov.il/api/Public...
3,215,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2331160,xlsx,1,71000001,2022 רבעון 3,0,300003,0,0,0,NaN,NaN,0,520032269,NaN,NaN,01/07/2022,31/10/2022,גמל,0,674058,1,NaN,0,"החברה לניהול קרן ההשתלמות לעובדי המדינה בע""מ",NaN,520032269,NaN,False,https://employersinfocmp.cma.gov.il/api/Public...
4,169,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2331604,xlsx,1,71000001,2022 רבעון 3,0,300003,0,0,0,NaN,NaN,0,520042581,NaN,NaN,01/07/2022,31/10/2022,גמל,0,674192,1,NaN,0,"עמ""י - חברה לניהול קופות גמל ענפיות בע""מ",NaN,520042581,NaN,False,https://employersinfocmp.cma.gov.il/api/Public...


# Process multiple reports -> unified file

In [9]:
reports_fn_list = get_filename_list(response_path)

number of files to be pre-processed: 66


## Pre-process - check sheet names and column names, add to standardization if needed

In [10]:
pre_process_reports(reports_fn_list)

Something went wrong with report: data/downloaded reports/company reports/2022Q3/2333675.xlsx


/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


{'סכום נכסי הקרן': 65, 'מזומנים': 65, 'תעודות התחייבות ממשלתיות': 65, 'תעודות חוב מסחריות': 65, 'אג"ח קונצרני': 65, 'מניות': 65, 'קרנות סל': 65, 'קרנות נאמנות': 65, 'כתבי אופציה': 65, 'אופציות': 65, 'חוזים עתידיים': 65, 'מוצרים מובנים': 65, 'לא סחיר - תעודות התחייבות ממשלתי': 65, 'לא סחיר - תעודות חוב מסחריות': 65, 'לא סחיר - אג"ח קונצרני': 65, 'לא סחיר - מניות': 65, 'לא סחיר - קרנות השקעה': 65, 'לא סחיר - כתבי אופציה': 65, 'לא סחיר - אופציות': 65, 'לא סחיר - חוזים עתידיים': 65, 'לא סחיר - מוצרים מובנים': 65, 'הלוואות': 65, 'פקדונות מעל 3 חודשים': 65, 'זכויות מקרקעין': 65, 'השקעה בחברות מוחזקות': 65, 'השקעות אחרות': 65, 'יתרת התחייבות להשקעה': 65, 'עלות מתואמת אג"ח קונצרני סחיר': 65, 'עלות מתואמת אג"ח קונצרני ל.סחיר': 65, 'עלות מתואמת מסגרות אשראי ללווים': 65, '{PL}PickLst': 3}


/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,מזומנים,תעודות התחייבות ממשלתיות,תעודות חוב מסחריות,"אג""ח קונצרני",מניות,קרנות סל,קרנות נאמנות,כתבי אופציה,אופציות,חוזים עתידיים,מוצרים מובנים,לא סחיר - תעודות התחייבות ממשלתי,לא סחיר - תעודות חוב מסחריות,"לא סחיר - אג""ח קונצרני",לא סחיר - מניות,לא סחיר - קרנות השקעה,לא סחיר - כתבי אופציה,לא סחיר - אופציות,לא סחיר - חוזים עתידיים,לא סחיר - מוצרים מובנים,הלוואות,פקדונות מעל 3 חודשים,זכויות מקרקעין,השקעה בחברות מוחזקות,השקעות אחרות,"עלות מתואמת אג""ח קונצרני סחיר","עלות מתואמת אג""ח קונצרני ל.סחיר",עלות מתואמת מסגרות אשראי ללווים,{PL}PickLst
שם המנפיק/שם נייר ערך,65.00,65.00,65.00,65.00,65.00,65.00,63.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,62.00,65.00,65.00,65.00,65.00,65.00,NaN
"מספר ני""ע",65.00,65.00,65.00,65.00,65.00,65.00,63.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,NaN,NaN,65.00,65.00,65.00,65.00,NaN
מספר מנפיק,65.00,NaN,65.00,65.00,65.00,65.00,63.00,NaN,NaN,NaN,NaN,NaN,65.00,65.00,65.00,NaN,NaN,NaN,NaN,NaN,65.00,65.00,NaN,65.00,NaN,NaN,NaN,NaN,NaN
דירוג,65.00,65.00,65.00,65.00,NaN,NaN,63.00,NaN,NaN,NaN,65.00,65.00,65.00,65.00,NaN,NaN,NaN,NaN,NaN,65.00,65.00,65.00,NaN,65.00,65.00,65.00,65.00,65.00,NaN
שם מדרג,65.00,65.00,65.00,65.00,NaN,NaN,63.00,NaN,NaN,NaN,65.00,65.00,65.00,65.00,NaN,NaN,NaN,NaN,NaN,65.00,65.00,65.00,NaN,65.00,65.00,65.00,65.00,65.00,NaN
סוג מטבע,65.00,65.00,65.00,65.00,65.00,65.00,63.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,62.00,65.00,65.00,65.00,65.00,65.00,NaN
שעור ריבית,65.00,65.00,65.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,65.00,65.00,65.00,65.00,NaN,NaN,NaN,NaN,NaN,65.00,65.00,65.00,NaN,65.00,65.00,65.00,65.00,65.00,NaN
תשואה לפדיון,65.00,65.00,65.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,65.00,65.00,65.00,65.00,NaN,NaN,NaN,NaN,NaN,65.00,65.00,65.00,NaN,65.00,65.00,NaN,NaN,NaN,NaN
שווי,65.00,65.00,65.00,65.00,65.00,65.00,63.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,62.00,65.00,65.00,65.00,65.00,65.00,NaN
שעור מנכסי אפיק ההשקעה,64.00,65.00,65.00,65.00,65.00,65.00,63.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,65.00,62.00,65.00,65.00,65.00,65.00,65.00,NaN


In [11]:
# TODO: look into reports with errors. some excel files get 'TypeError'
# fn = response_path + "2287555.xlsx"
# report = pd.read_excel(fn)

# process holding reports
1. summary sheets (for verification)
2. holding sheets - raw data

## 1. summary sheets

In [12]:
all_summary_sheets = process_summary_sheets(reports_fn_list)

Something went wrong with report: data/downloaded reports/company reports/2022Q3/2333675.xlsx


/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:301: FutureWarning: The default value of regex will change from True to False in a future version.
  all_summary_sheets["pct_num"] = all_summary_sheets["pct"].astype(str).str.replace(r'[\%\s-]', '')


In [13]:
totals = get_totals(all_summary_sheets)

Number of totals found: 65


In [14]:
totals["sum_num"].sum()

1693918962.3105307

## 2. Holdings

In [15]:
# extract all holdings from reports
all_holdings_clean = clean_holdings(extract_holdings(reports_fn_list))

Something went wrong with report: data/downloaded reports/company reports/2022Q3/2333675.xlsx


/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing report 65 out of 66
before cleaning: 93823
 after cleaning: 83962


# add fund and company data to holdings

In [18]:
## Manually add missing corps to reports - only if needed, for funds processing
missing_corps = pd.DataFrame(columns=reports.columns)
# missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
#                                       'ParentCorpName': 'מנורה מבטחים ביטוח בע"מ',
#                                       'ParentCorpLegalId': '520042540',
#                                       'ReportPeriodDesc': ''
#                                      }, ignore_index=True)
# missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
#                                       'ParentCorpName': 'כלל חברה לביטוח בע"מ',
#                                       'ParentCorpLegalId': '520024647',
#                                       'ReportPeriodDesc': ''
#                                      }, ignore_index=True)

# add infinity
# missing_corps = missing_corps.append({'SystemName': 'גמל',
#                                       'ParentCorpName': 'אינפיניטי השתלמות, גמל ופנסיה בע"מ',
#                                       'ParentCorpLegalId': '513621110',
#                                       'ReportPeriodDesc': ''
#                                      }, ignore_index=True)

missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
                                      'ParentCorpName': 'הפניקס חברה לביטוח בע"מ',
                                      'ParentCorpLegalId': '520023185',
                                      'ReportPeriodDesc': ''
                                     }, ignore_index=True)

reports = reports.append(missing_corps)

/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_7595/1467351321.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_7595/1467351321.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reports = reports.append(missing_corps)


In [19]:
all_holdings_final = add_report_data(all_holdings_clean, reports)

/Users/urimarom/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


# Compare totals from summary sheets with totals from holdings

In [20]:
print("total from summary sheets: {:,.2f}".format(totals["sum_num"].sum()))
print("total from holdings sheets: {:,.2f}".format(all_holdings_final["שווי"].sum()))
print("diff: {:,.2f}".format(totals["sum_num"].sum() - all_holdings_final["שווי"].sum()))

total from summary sheets: 1,693,918,962.31
total from holdings sheets: 1,694,256,080.71
diff: -337,118.40


In [21]:
totals_from_holdings = all_holdings_final.groupby(["report_id", "ParentCorpName"])[["שווי"]].sum().reset_index()
totals_from_holdings.index = totals_from_holdings.index.astype('str')

In [22]:
totals_comparison = totals_from_holdings.merge(totals,
                                               on='report_id',
                                               how='outer'
                                              )

totals_comparison["diff"] = totals_comparison["sum_num"] - totals_comparison["שווי"]
totals_comparison[abs(totals_comparison["diff"]) > 1].sort_values("diff", ascending=False)

,report_id,ParentCorpName,שווי,asset,sum,pct,pct_num,sum_num,diff
54,2353564,"אלטשולר שחם גמל ופנסיה בע""מ","152,116,054.49","סה""כ סכום נכסי המסלול או הקרן","152,146,131.33",1,1,"152,146,131.33","30,076.84"
0,2329756,קופת תגמולים של עובדי אל על נתיבי אוויר לישראל...,"746,248.82",סה''כ סכום נכסי המסלול או הקרן,"749,194.73",1,1,"749,194.73","2,945.91"
35,2349946,"כלל פנסיה וגמל בע""מ","51,593,310.54","סה""כ סכום נכסי המסלול או הקרן","51,593,721.21",1.00,0.9999999999999997,"51,593,721.21",410.68
64,2356225,"הפניקס פנסיה וגמל בע""מ","57,642,854.55","סה""כ סכום נכסי המסלול או הקרן","57,635,078.87",1,1,"57,635,078.87","-7,775.68"
62,2356076,"הפניקס חברה לביטוח בע""מ","77,095,216.72","סה""כ סכום נכסי המסלול או הקרן","77,086,420.78",1,1,"77,086,420.78","-8,795.94"
10,2334569,"חברת הגמל לעובדי האוניברסיטה העברית בע""מ","2,656,764.93","סה""כ סכום נכסי המסלול או הקרן","2,302,782.83",1,1,"2,302,782.83","-353,982.10"


In [23]:
# TODO: add comparison by holding type (summary vs. sheets)
# report drill down
def report_sums_by_type(report_id):
    rep = all_holdings_final[all_holdings_final["report_id"] == report_id]
    print("total sum by holdings: {:,.2f}".format(rep["שווי"].sum()))
    return pd.DataFrame(rep.groupby("holding_type")["שווי"].agg('sum').map('{:,.2f}'.format))
    
report_sums_by_type('2353564')

total sum by holdings: 152,116,054.49


,שווי
holding_type,
"אג""ח קונצרני","6,398,982.75"
אופציות,"-7,009.06"
הלוואות,"14,079,221.58"
השקעות אחרות,"26,335.19"
זכויות מקרקעין,"506,713.82"
חוזים עתידיים,"-2,099,707.22"
כתבי אופציה,"19,964.24"
"לא סחיר - אג""ח קונצרני","2,362,440.70"
לא סחיר - חוזים עתידיים,"-2,317,999.22"


In [24]:
all_summary_sheets[
    all_summary_sheets["report_id"] == '2353564'
]

,asset,sum,pct,report_id,pct_num,sum_num
1429,א. מזומנים,"19,710,396.99",0.13,2353564,0.1295,"19,710,396.99"
1430,ב. ניירות ערך סחירים:,NaN,NaN,2353564,nan,NaN
1431,(1) תעודות התחייבות ממשלתיות,"55,790,188.23",0.37,2353564,0.3667,"55,790,188.23"
1432,(2) תעודות חוב מסחריות,0,0,2353564,0,0.00
1433,"(3) אג""ח קונצרני","6,398,982.75",0.04,2353564,0.0421,"6,398,982.75"
1434,(4) מניות,"26,061,254.54",0.17,2353564,0.1713,"26,061,254.54"
1435,(5) קרנות סל,"3,843,150.29",0.03,2353564,0.0253,"3,843,150.29"
1436,(6) תעודות השתתפות בקרנות נאמנות,"3,591,452.97",0.02,2353564,0.0236,"3,591,452.97"
1437,(7) כתבי אופציה,"19,964.24",0.00,2353564,0.0001,"19,964.24"
1438,(8) אופציות,"-7,009.06",0,2353564,0,"-7,009.06"


### totals discrepancies
<u>2021Q2</u><br>
* In report 2128339 - there's a redundant line in נדלן, consider removing manually when all files for Q3 are here - removing
* In report 2153994 - the holdings in מזומנים are mostly missing. adding an artificial row. (הפניקס)
* In report 2146293 - the value of holdings in לא סחיר - אג"ח קונצרני are way off. leaving as is for now.

<u>2021Q3</u>
* 2148642 - missing exactly 10000 in מזומנים. leaving as is.
* 2148994 - dropped a line in הלוואות withno security number. not fixing for now.

<u>2021Q3</u>
* 2222108 - the holdings in מזומנים are mostly missing. adding an artificial row. (הפניקס)

<u>2022Q1</u>
* 2235319 - נדלן. לא תוקן (לא משתתף במדרג)
* 2260140 - הפניקס פנסיה. הבדלים במזומנים. משאיר ככה כרגע.
* 2256493 - missing one holding in mutual funds, 50M

<u>2022Q2</u>
* 2308537 - הפניקס פנסיה. הבדלים במזומנים. משאיר ככה כרגע.

## Removing redundant rows

In [ ]:
# remove_rows = (
#     (all_holdings_final["report_id"] == '2128339') &
#     (all_holdings_final["holding_type"] == 'זכויות מקרקעין') &
#     (all_holdings_final["שעור מנכסי אפיק ההשקעה"] == 1)
# )

all_holdings_final = all_holdings_final.drop(all_holdings_final[remove_rows].index, axis=0)

# all_holdings_final.groupby("report_id").filter(
#     lambda x: x["שעור מנכסי אפיק ההשקעה"].sum() > 1
# ).head()

## Adding missing rows

In [ ]:
# adding a row for report 2153994, to compensate for missing holding values
total_from_summary = all_summary_sheets[
    (all_summary_sheets["report_id"] == '2222108') &
    (all_summary_sheets["asset"]).str.contains('מזומנים')
]["sum_num"]

holdings_to_fix = all_holdings_final[
    (all_holdings_final["report_id"] == '2222108') &
    (all_holdings_final["holding_type"] == 'מזומנים')
]

total_from_holdings = holdings_to_fix["שווי"].sum()

# calculate sum and pct to be added
to_be_added_sum = float(total_from_summary) - float(total_from_holdings)
to_be_added_pct = 1 - holdings_to_fix["שעור מנכסי אפיק ההשקעה"].sum()
to_be_added_pct_of_report = float(to_be_added_sum) / float(totals[totals["report_id"] == '2222108']["sum_num"])

# prepare new row - take the highest amount row as base and edit it
new_row = holdings_to_fix.sort_values("שווי", ascending=False).head(1)
new_row["שם המנפיק/שם נייר ערך"] = "unknown - added to match sum"
new_row['שווי'] = to_be_added_sum
new_row['שעור מנכסי אפיק ההשקעה'] = to_be_added_pct
new_row['שעור מסך נכסי השקעה'] = to_be_added_pct_of_report

for col in ['מספר ני"ע', 'מספר מנפיק', 'דירוג', 'שם מדרג', 'סוג מטבע']:
    new_row[col] = None

print("adding row: ")
new_row

In [ ]:
# adding the artificial row
all_holdings_final = all_holdings_final.append(new_row)

In [25]:
all_holdings_final.to_csv(response_path+"all_holdings.csv", index=False)

In [26]:
### uncomment when running on fund reports ###
# all_holdings_ever = all_holdings_final

## Export file for classification

In [27]:
cols_for_cls = [
    'שם המנפיק/שם נייר ערך', 'מספר ני"ע', 'מספר מנפיק',
    'שווי' ,'שעור מנכסי אפיק ההשקעה', 'שעור מסך נכסי השקעה', 
       'holding_type', 'זירת מסחר', 'תאריך רכישה', 'ערך נקוב', 'שער','שעור מערך נקוב מונפק',
       'ענף מסחר',
       'SystemName', 'ParentCorpName', 'ReportPeriodDesc'
]

holdings_for_cls = all_holdings_final[
    all_holdings_final["holding_type"].isin(['מניות', 'אג"ח קונצרני'])
]

holdings_for_cls[cols_for_cls].to_csv(response_path+"holdings_for_cls.csv", index=False)

In [ ]:
# holdings_for_cls[
#     (holdings_for_cls['מספר ני"ע'].astype(str) == '11616780')
# ]

## add new processed holdings to all_holdings file

In [ ]:
fetch_all_holdings_path()

In [28]:
# reload results from file
all_holdings_path = fetch_all_holdings_path()
new_holdings_path = response_path+"all_holdings.csv"
all_holdings_ever = concat_from_csv_by_path(all_holdings_path, new_holdings_path)

pd.DataFrame(all_holdings_ever.groupby(['ReportPeriodDesc'])['שווי'].agg('sum').map('{:,.2f}'.format))

/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:582: DtypeWarning: Columns (6,7,13,20,22,23,24,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  all_holdings = pd.read_csv(all_holdings_path, dtype=holdings_dtypes())
/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:583: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  new_holdings = pd.read_csv(new_holdings_path, dtype=holdings_dtypes())


,שווי
ReportPeriodDesc,
2020 רבעון 1,"1,222,418,614.88"
2020 רבעון 2,"1,307,193,119.07"
2020 רבעון 3,"1,373,544,876.41"
2020 רבעון 4,"1,461,378,236.82"
2021 רבעון 1,"1,548,168,300.50"
2021 רבעון 2,"1,593,940,074.16"
2021 רבעון 3,"1,640,889,505.45"
2021 רבעון 4,"1,776,286,225.46"
2022 רבעון 1,"1,778,911,368.39"


In [ ]:
# update all holdings file - only for companies! - handle with care
# TODO: add backup process
# all_holdings_ever.to_csv(fetch_all_holdings_path(), index=False)

<h1><center>*** END OF PART 1 ***</center></h1>

# Validate holdings file

In [29]:
all_holdings_curr_q = all_holdings_ever[all_holdings_ever["ReportPeriodDesc"] == '2022 רבעון 3']
# pd.DataFrame(all_holdings_curr_q.groupby(['SystemName', 'ParentCorpName', 'report_id'])['שווי'].agg('sum').map('{:,.2f}'.format))
pd.DataFrame(all_holdings_curr_q.groupby(['ParentCorpName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
ParentCorpName,
"איי. די. איי. חברה לביטוח בע""מ","300,804.50"
"איילון חברה לביטוח בע""מ","4,532,465.55"
"אינפיניטי השתלמות, גמל ופנסיה בע""מ","1,537,738.23"
"אלטשולר שחם גמל ופנסיה בע""מ","178,717,896.71"
"אנליסט קופות גמל בע""מ","19,001,666.27"
ארם גמולים - חברה לניהול קופות גמל בע''מ,"1,056,016.94"
"גל -ניהול קופות גמל לעובדי הוראה בע""מ","4,284,315.11"
"החברה המנהלת של מינהל קרן ההשתלמות לפקידים עובדי המנהל והשירותים בע""מ","830,031.07"
"החברה המנהלת של קרן השתלמות של עובדי חברת החשמל לישראל בע""מ","631,134.72"


## Look for missing company reports

In [ ]:
reports_cnt = pd.DataFrame(all_holdings_ever.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId'])['ReportPeriodDesc'].nunique().reset_index())
period_cnt = all_holdings_ever["ReportPeriodDesc"].nunique()

companies_with_missing_reports = reports_cnt[reports_cnt["ReportPeriodDesc"] < period_cnt]

In [ ]:
available_reports_periods = pd.DataFrame(all_holdings_ever.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId', 'ReportPeriodDesc'])['שווי'].sum()).reset_index()
companies_w_missing_reports_available_reports = pd.merge(
    companies_with_missing_reports[['ParentCorpLegalId', 'SystemName']],
    available_reports_periods,
    on = ['ParentCorpLegalId', 'SystemName'],
    how="inner"
)
companies_w_missing_reports_available_reports

Located missing reports for major companies, manually downloaded

## Look for missing fund reports

### 1. Using company reports

In [ ]:
quarter = '2022Q1'
funds_path = "data/downloaded reports/fund reports " + quarter + "/all_holdings.csv"
company_path = "data/downloaded reports/company reports " + quarter + "/all_holdings.csv"

id_dtypes={"report_id":str, "ParentCorpLegalId":str, "ProductNum":str}

funds_holdings = pd.read_csv(funds_path, dtype=id_dtypes)
# change חיים ואובדן כושר עבודה to ביטוח
funds_holdings["SystemName"] = funds_holdings["SystemName"].str.replace("חיים ואובדן כושר עבודה", "ביטוח")
company_holdings = pd.read_csv(company_path, dtype=id_dtypes)

In [ ]:
funds_agg = funds_holdings.groupby(["ParentCorpName", "ParentCorpLegalId", "SystemName"])[["שווי"]].sum().reset_index()
company_agg = company_holdings.groupby(["ParentCorpName", "ParentCorpLegalId", "SystemName"])[["שווי"]].sum().reset_index()

comparison = pd.merge(left=company_agg,
                      right=funds_agg,
                      how='outer',
                      on=["ParentCorpLegalId", "SystemName"],
                      suffixes=['', '_funds']
                     )
comparison["ParentCorpName"] = comparison["ParentCorpName"].fillna(comparison["ParentCorpName_funds"])
comparison.drop("ParentCorpName_funds", axis=1, inplace=True)
comparison["sum_diff"] = comparison["שווי"] - comparison["שווי_funds"]

companies_missing_funds = comparison[(abs(comparison["sum_diff"]) > 1) |
                                     (comparison["sum_diff"].isna())
                                    ].sort_values("sum_diff", ascending=False)
companies_missing_funds

In [ ]:
funds_agg_by_product = funds_holdings.groupby([
    "ParentCorpName",
    "SystemName",
    "ProductNum",
    "ShortName",
    "report_id",
    "ParentCorpLegalId"
], dropna=False)[["שווי"]].sum().reset_index()

pd.merge(left=companies_missing_funds,
         right=funds_agg_by_product,
         how='left',
         on=['ParentCorpLegalId', "SystemName"]
        )
# .sort_values(["sum_diff", "ProductNum"], ascending=False)

### 2. using pensia-net, bituach-net, gemel-net
API calls to x-net gov site:
https://data.gov.il/dataset/pensia-net


In [ ]:
import urllib.request as urlreq

def x_net_types():
    return {
    "DEPOSITS": float,
    "WITHDRAWLS": float,
    "INTERNAL_TRANSFERS": float,
    "NET_MONTHLY_DEPOSITS": float,
    "TOTAL_ASSETS": float,
    "AVG_ANNUAL_MANAGEMENT_FEE": float,
    "AVG_DEPOSIT_FEE": float,
    "MONTHLY_YIELD": float,
    "YEAR_TO_DATE_YIELD": float,
    "ACTUARIAL_ADJUSTMENT": float,
    "YIELD_TRAILING_3_YRS": float, 
    "YIELD_TRAILING_5_YRS": float,
    "AVG_ANNUAL_YIELD_TRAILING_3YRS": float,
    "AVG_ANNUAL_YIELD_TRAILING_5YRS": float,
    "STANDARD_DEVIATION": float,
    "ALPHA": float,
    "SHARPE_RATIO": float,
    "LIQUID_ASSETS_PERCENT": float,
    "STOCK_MARKET_EXPOSURE": float,
    "FOREIGN_EXPOSURE": float,
    "FOREIGN_CURRENCY_EXPOSURE": float
}

def read_from_x_net(resource_id):
    url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=' + resource_id + '&limit=32000'
    fileobj = urlreq.urlopen(url)
    df = pd.DataFrame(pd.read_json(fileobj)["result"]["records"])
    # handle dtypes
    for c in x_net_types():
        if c in df:
            df[c] = df[c].astype(x_net_types()[c], errors='ignore')
    return df

pensia_net = read_from_x_net('6d47d6b5-cb08-488b-b333-f1e717b1e1bd')
pensia_net['SystemName'] = 'פנסיה'
bituach_net = read_from_x_net('c6c62cc7-fe02-4b18-8f3e-813abfbb4647')
bituach_net['SystemName'] = 'ביטוח'
gemel_net = read_from_x_net('a30dcbea-a1d2-482c-ae29-8f781f5025fb')
gemel_net['SystemName'] = 'גמל'

x_net = pd.concat([pensia_net, bituach_net, gemel_net])

In [ ]:
# leave only relevant data
first_month_of_q = (int(quarter[-1:])) * 3 - 2
last_month_of_q = first_month_of_q + 2
year = quarter[0:4]
# yearmonth = year + '0' + str(first_month_of_q)
yearmonth = year + '0' + str(last_month_of_q)

# x_net_net_first_month_of_q = x_net[x_net['REPORT_PERIOD'] == yearmonth]
x_net_net_last_month_of_q = x_net[x_net['REPORT_PERIOD'] == yearmonth]

funds = pd.merge(left=x_net_net_last_month_of_q,
                 right=funds_agg_by_product,
                 how='outer',
                 left_on=['MANAGING_CORPORATION_LEGAL_ID', 'FUND_ID'],
                 right_on=['ParentCorpLegalId', "ProductNum"],
                 suffixes=['_x_net', '']
                )
funds["SystemName"] = funds["SystemName"].fillna(funds["SystemName_x_net"])
funds.drop("SystemName_x_net", axis=1, inplace=True)
pd.crosstab(funds["SystemName"], funds["report_id"].notnull())

In [ ]:
missing_funds = funds[(funds["report_id"].isnull()) | (funds["FUND_ID"].isnull())]
missing_funds.head()

In [ ]:
# aggregate x_net data per company
funds.groupby(["MANAGING_CORPORATION_LEGAL_ID", "ParentCorpName", "SystemName"])[["TOTAL_ASSETS", "שווי"]].sum()

<h1><center>*** Add fossil classification ***</center></h1>
Based on quarterly company level reports classification

## get previous fossil classifications

In [34]:
prev_cls_fn = "data_sources/prev_class.csv"
prev_by_sec_num, prev_by_ISIN = get_latest_fossil_classifications(prev_cls_fn)

/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:507: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_csv = pd.read_csv(prev_cls_fn, parse_dates=['classification_date'])


previously classified Israeli security nums by is_fossil:
0.00    2543
1.00     258
NaN        1
Name: is_fossil, dtype: int64
previously classified ISINs by is_fossil:
0.00    7087
1.00     605
Name: is_fossil, dtype: int64


## directly using previous company reports
Could be useful for fund level reports, commented out for now

In [ ]:
# # TODO: using different file per holding_type for now, should change
# q_company_report1 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות באגח קונצרני.csv')
# q_company_report2 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות במניות.csv')

# q_company_report = pd.concat([
#     q_company_report1[['מספר ני"ע', 'is_fossil']],
#     q_company_report2[['מספר ני"ע', 'is_fossil']]
# ])

# q_company_report = q_company_report.drop_duplicates(subset=['מספר ני"ע']).set_index('מספר ני"ע')
# q_company_report.index = q_company_report.index.astype('str')

## Using previous classification

In [ ]:
# don't run when processing fund reports
all_holdings_ever = pd.read_csv(fetch_all_holdings_path(), dtype=str)

In [35]:
all_holdings_cls = add_fossil_classifications(all_holdings_ever, prev_by_sec_num, prev_by_ISIN)

all_holdings: 919370
having holding number: 776755
without holding number: 142615


/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:544: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holdings_with_num['מספר ני"ע'] = holdings_with_num['מספר ני"ע'].astype('str').str.strip().str.upper()


Holdings after fossil classification by Israeli security num:
NaN     537552
0.00    209164
1.00     30039
Name: is_fossil, dtype: int64
Holdings after fossil classification by ISIN:
NaN     424189
0.00    314527
1.00     38039
Name: is_fossil, dtype: int64
total fossil sum: 625408362.7655122
holdings count before classification: 919370
holdings count after classification: 919370


## Validate Fossil Classification results

In [36]:
# look for fossil holdings that aren't מניות or אג"ח קונצרני
all_holdings_cls[
    (all_holdings_cls["is_fossil"] > 0) &
    (~all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני']))
]

# looking good!
# TODO: might want to try and catch additional fossil holdings of other types - join by all id types

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי
3920,Israel electric 4% 19/06/28- חברת החשמל לישראל...,XS0085848421,520000472,BBB,S&P,יין יפני,0.04,0.01,"138,447.62",0.05,0.00,2020739,"לא סחיר - אג""ח קונצרני",NaN,04/08/15,6.61,"3,768,000,000.00",117.80,NaN,0.00,אחר,Utilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אלטשולר שחם גמל ופנסיה בע""מ",513173393,0.0,NaN,NaN,07/04/2021,2020 רבעון 4,1.00,"138,447.62"
4903,נאוויטס פט אפ 4,1156454,NaN,NaN,NaN,שקל חדש,NaN,NaN,71.29,0.41,0.00,1823174,כתבי אופציה,TASE,NaN,NaN,"36,750.00",194.00,NaN,0.03,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,החברה המנהלת של רום קרן ההשתלמות לעובדי הרשויו...,520031824,0.0,NaN,NaN,25/05/2020,2020 רבעון 1,1.00,71.29
7257,רציו אפ 19,3940319,NaN,NaN,NaN,שקל חדש,NaN,NaN,32.16,0.00,0.00,2056812,כתבי אופציה,TASE,NaN,NaN,"417,634.56",7.70,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ביטוח,"הראל חברה לביטוח בע""מ",520004078,0.0,NaN,NaN,09/06/2021,2021 רבעון 1,1.00,32.16
8423,נאוויטס פט אפ 4,1156454,NaN,NaN,NaN,שקל חדש,NaN,NaN,8.55,0.01,0.00,1944444,כתבי אופציה,TASE,NaN,NaN,"23,300.00",36.70,NaN,0.02,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.0,NaN,NaN,29/11/2020,2020 רבעון 3,1.00,8.55
8424,רותם אנרגיה אפ1,1155720,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.20,0.00,0.00,1944444,כתבי אופציה,TASE,NaN,NaN,"9,050.00",2.20,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.0,NaN,NaN,29/11/2020,2020 רבעון 3,1.00,0.20
8425,רותם אנרגיה אפ2,1155738,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.85,0.00,0.00,1944444,כתבי אופציה,TASE,NaN,NaN,"7,240.00",11.70,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.0,NaN,NaN,29/11/2020,2020 רבעון 3,1.00,0.85
8426,רותם אנרגיה אפ3,1155746,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.31,0.00,0.00,1944444,כתבי אופציה,TASE,NaN,NaN,"1,810.00",17.00,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.0,NaN,NaN,29/11/2020,2020 רבעון 3,1.00,0.31
10179,XOP US,US78468R5569,513834200,NaN,NaN,דולר אמריקאי,NaN,NaN,"20,532.32",0.01,0.00,2106482,קרנות סל,NASDAQ,NaN,NaN,"65,138.67","9,669.00",0.00,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",512227265,0.0,NaN,NaN,09/09/2021,2021 רבעון 2,1.00,"20,532.32"
10274,רציו אפ 19,3940319,NaN,NaN,NaN,שקל חדש,NaN,NaN,5.99,0.00,0.00,2106482,כתבי אופציה,TASE,NaN,NaN,"598,810.00",1.00,NaN,0.01,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",512227265,0.0,NaN,NaN,09/09/2021,2021 רבעון 2,1.00,5.99
10911,ORGS US - אופ לא סחירה 20.1.23,707705711,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,65.63,0.05,0.00,2106482,לא סחיר - אופציות,NaN,31/05/2021,NaN,"26,490.26",76.00,NaN,0.00,NaN,ל.ר.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",512227265,0.0,NaN,NaN,09/09/2021,2021 רבעון 2,1.00,65.63


In [37]:
should_be_cls = all_holdings_cls[all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני'])]
missing_fossil_cls = pd.DataFrame(
    should_be_cls["is_fossil"].isnull().groupby([should_be_cls["ParentCorpName"], should_be_cls["ReportPeriodDesc"]]).sum()
)
missing_fossil_cls

is_fossil
ParentCorpName                                     ReportPeriodDesc           
איי. די. איי. חברה לביטוח בע"מ                     2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
                                                   2022 רבעון 1              0
                                                   2022 רבעון 2              0
                                                   2022 רבעון 3              0
איילון חברה לביטוח בע"מ                            2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
                                                   2022 רבעון 1              0
                                                   2022 רבעון 2              0
                                                   2022 רבעון 3              0
אינפיניטי השתלמות, גמל ופנסיה בע"מ                 2022 רבעון 2              0
                                                   2022 רבעון 3              0
אינפיניטי ניהול השתלמות וגמל בע"מ                  2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
אלטשולר שחם גמל ופנסיה בע"מ                        2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
                                                   2022 רבעון 1              0
                                                   2022 רבעון 2              0
                                                   2022 רבעון 3              0
אנליסט קופות גמל בע"מ                              2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                        

## Write classified results to CSV & create another file with added id types

In [38]:
all_holdings_cls.to_csv(response_path+"all_holdings_cls.csv", index=False)
# update all_holdings_cls_all_ids if needed
all_holdings_cls_all_ids = load_mappings_and_add_ids_to_holdings(all_holdings_cls)
all_holdings_cls_all_ids.to_csv(response_path+"all_holdings_cls_all_ids.csv", index=False)

/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:128: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cnt = sum(df[col].astype(str).str.strip().str.contains(pattern, na=False))



Holding file ISIN col is: מספר ני"ע
number of ISINs: 142766 out of 919370 rows

Holding file מספר תאגיד col is: מספר מנפיק
number of מספר תאגידs: 300608 out of 919370 rows

no LEIs in holdings file


/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:165: FutureWarning: The default value of regex will change from True to False in a future version.
  df[id_cols[id_type]] = df[id_cols[id_type]].str.replace(id_col_patterns(id_type), "")
/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:167: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matching_type = df[id_type].str.contains(id_col_patterns(id_type), na=False)
/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:165: FutureWarning: The default value of regex will change from True to False in a future version.
  df[id_cols[id_type]] = df[id_cols[id_type]].str.replace(id_col_patterns(id_type), "")
/Users/urimarom/PycharmProjects/fossil_classification/enrich_holdings.py:167: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the grou

מספר ני"עs with matching ISIN: 301775 out of total relevant rows: 772724
מספר תאגידs with matching מספר מנפיק: 287801 out of total relevant rows: 300608
מספר ני"עs with matching מספר מנפיק: 290003 out of total relevant rows: 772724
ISINs with matching מספר מנפיק: 290920 out of total relevant rows: 301775
ISINs with matching LEI: 73232 out of total relevant rows: 301775


In [ ]:
# additional output formats
all_holdings_cls_all_ids.to_pickle(response_path+"all_holdings_cls_all_ids.pkl", compression='gzip')
all_holdings_cls_all_ids.to_json(response_path+"all_holdings_cls_all_ids.json", compression='gzip')

## Get Fund stats

In [ ]:
all_holdings_cls_all_ids["שווי"] = all_holdings_cls_all_ids["שווי"].astype(float)

In [ ]:
fund_stats = get_summary(all_holdings_cls_all_ids, 'ParentCorpName', 'SystemName', 'ProductNum','ShortName')
fund_stats.sort_values('שווי פוסילי', ascending=False).head()

In [ ]:
fund_stats.to_csv(response_path+"fund_stats.csv", index=False)

In [ ]:
# get fossil holdings per fund

fund_stats[fund_stats["ProductNum"].duplicated()]
# TODO: there is 1 duplicate, fund number 103 - check it later
def get_fossil_holdings(fund_id):
    fossil_holdings = all_holdings_cls[
        (all_holdings_cls["ProductNum"] == fund_id) &
        (all_holdings_cls["is_fossil"] > 0)
    ]
    return fossil_holdings.sort_values("שווי פוסילי", ascending=False)

get_fossil_holdings(11366)

## חיסכון לכל ילד

In [8]:
fund_stats = pd.read_csv(response_path+"fund_stats.csv")

In [10]:
hly = fund_stats[
    fund_stats["ShortName"].str.contains("ילד", na=False)
].sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים", ascending=False).reset_index()
hly.to_csv(response_path+"hly_fund_stats.csv", index=False)
hly

,index,ParentCorpName,SystemName,ProductNum,ShortName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים,שווי בסוגי החזקות לא פוסיליים,שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים
0,580,"מור גמל ופנסיה בע""מ",גמל,9303,מור חיסכון לילד - שריעה,"13,160.23",662.30,0.05,"2,760.75",662.30,0.24,"7,445.19",0.06
1,100,"אנליסט קופות גמל בע""מ",גמל,11366,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,"109,359.64","10,153.11",0.09,"48,612.62","10,153.11",0.21,"32,876.77",0.12
2,99,"אנליסט קופות גמל בע""מ",גמל,11365,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון מועט,"321,891.69","18,553.97",0.06,"112,444.37","18,553.97",0.17,"154,760.48",0.07
3,760,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11321,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"374,546.50","15,065.45",0.04,"93,053.47","15,065.45",0.16,"163,515.84",0.06
4,532,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,9898,מגדל חסכון לילד - חוסכים המעדיפים סיכון מוגבר,"59,704.38","3,229.68",0.05,"21,202.13","3,229.68",0.15,"14,775.00",0.09
5,231,"הפניקס פנסיה וגמל בע""מ",גמל,11310,הפניקס חיסכון לילד - חוסכים המעדיפים סיכון מועט,"646,914.63","45,269.51",0.07,"301,948.96","45,269.51",0.15,"259,819.30",0.08
6,531,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,9897,מגדל חסכון לילד - חוסכים המעדיפים סיכון בינוני,"80,695.81","4,650.18",0.06,"31,201.69","4,650.18",0.15,"20,638.10",0.09
7,530,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,9896,מגדל חסכון לילד - חוסכים המעדיפים סיכון מועט,"362,358.49","18,966.67",0.05,"128,645.27","18,966.67",0.15,"119,269.33",0.08
8,761,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11322,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"105,466.81","5,436.38",0.05,"38,550.78","5,436.38",0.14,"25,938.12",0.08
9,333,"הראל פנסיה וגמל בע""מ",גמל,11377,הראל חיסכון לילד - חוסכים המעדיפים סיכון מוגבר,"466,323.38","29,728.13",0.06,"209,639.62","29,354.26",0.14,"94,328.43",0.10


In [ ]:
fund_stats.groupby(["ParentCorpName"]).count()

In [ ]:
all_holdings_cls.groupby(["SystemName","ParentCorpName","ProductNum"])["שעור מסך נכסי השקעה"].sum()

## Fund level data - Discrepancies
* הלמן - seems like we looked at the wrong company level report - it was חברת חשמל...
<br>The numbers ~fit the ones in the company level PDF report. The gov site is missing a company level report.

the company name is: הלמן-אלדובי קופות גמל ופנסיה בע"מ


In [ ]:
# missing reports
[r for r in funds_df.index.unique() if r not in all_holdings_final["report_id"].unique()]

# checked 2 missing, both have 0 holdings, all good

In [ ]:
pd.DataFrame(all_holdings_final.groupby(['ParentCorpName', 'SystemName'])["report_id"].nunique())

### discrepancies

* report 2101262: empty sheet "זכויות מקרקעין" - explains all the diff
* report 2101276: all holding_types sums are ok, the sum @summary sheet is slightly different for some reason

In [ ]:
print(funds_df[funds_df["ProductNum"] == 2207])
totals[totals["report_id"] == '2101262']

# looking for lost insurance funds

### Migdal

In [ ]:
migdal_ins = all_holdings_final[(all_holdings_final["ParentCorpLegalId"] == '520004896')]
migdal_by_fund = pd.DataFrame(migdal_ins.groupby(["ShortName", "ProductNum", "report_id"], dropna=False)["שווי"].agg('sum'))
migdal_by_fund.reset_index()

https://www.migdal.co.il/about/investments/assets - there are 4 more funds not included in the reports site:
* מגדל משתתף ברווחים - קרן ח - b010_p221_520004896
* מגדל משתתף ברווחים - קרן ט - 520004896_b011_p221
* מגדל משתתף ברווחים - קרן י - b012_p221_520004896
* מגדל משתתף קרן י החדשה - b013_p221_520004896

Manually downloaded, added to the directory.

In [ ]:
pd.DataFrame(all_holdings_final[
    (all_holdings_final["ParentCorpLegalId"].isnull()) |
    (all_holdings_final["ParentCorpLegalId"] == 520004896)
].groupby("report_id")['שווי'].agg('sum').map('{:,.2f}'.format))

### Menora missing reports

In [ ]:
pd.DataFrame(all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ'
].groupby(["ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)).sort_values("ProductNum").reset_index()

* total 17 funds in website, 2 of which mergerd into others --> 15.
<br><br>
<u>missing reports:</u>
1. 18011
1. 18012

download from here: https://www.menoramivt.co.il/rep-list/about-insurance/participating-portfolio-info/insurer-assets

In [ ]:
# adding sums of 2 migrated funds (data for these 2 is 1 quarter behind)
99576.34 + 531016.24 + all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ']["שווי"].sum()

Conclusion: there seems to be missing reports for Menora. Check again next quarter.

### הפניקס ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520023185'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

* website: 30 funds
* added 2 missing reports: 7012, 7011 (קרן ט, קרן י)
<br>
https://www.fnx.co.il/investors-relations-hebrew/investment-information-reports/reshimat-nichsey-hamevateah

In [ ]:
url = "https://cdn.menoramivt.co.il/public/docs/20211205/520042540_b18012_p321.xlsx"
filename = "520042540_b18012_p321.xlsx"
ur.urlretrieve(url, filename)

### הראל ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520004078'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

* website: 34 funds.<br><br>
<B> Manually added missing reports: 259010 259011 259012 259013 </B>
from website: https://www.harel-group.co.il/about/harel-group/harel/mesthtefet/Pages/list-of-assets.aspx

### כלל ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520024647'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

* website: 21 funds
* missing reports: 14011, 14012
<br><br>
downloaded from their site: https://www.clalbit.co.il/aboutclalinsurance/generalreports/

## הלמן - פנסיה וגמל
?גמל - הבדל בסכום, האם כתוצאה מדוחות חסרים
<br>
₪16,136,063 בדוח מסכם
16,132,489.84 לפי סכום דוחות מסלול

הבדלים קטנים, לא מצאתי הסבר.
אולי הוציאו דוחות בשעות שונות של היום?

In [ ]:
16136063 - 16132489.84

In [ ]:
helman = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("הלמן")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
helman_by_report = pd.DataFrame(helman.groupby(["ProductNum", "ShortName"])["שווי"].agg('sum').map('{:,.2f}'.format)).reset_index()
helman_by_report

## מגדל פנסיה

In [ ]:
migdal_p = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("מגדל")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
migdal_p_report = pd.DataFrame(migdal_p.groupby(["report_id", "ProductNum", "ShortName"])["שווי"].agg('sum')).reset_index()
migdal_p_report.merge(totals,
                     on="report_id",
                     how="left")

# migdal_p_report["שווי"].sum()

* All pension funds from website are on file.<br>
<B> last Q there was also a discrepancy (2.1B, now 2.15B) </B>